In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
import datetime as dt

import numpy as np
import pandas as pd

from deep_hedging import (
    Underlyings,
    Ticker,
    ConstantRateCurve,
    EuropeanCall,
    EuropeanPut,
)
from deep_hedging.monte_carlo import GBMPricer

RANDOM_SEED = 12

In [3]:
N_PATHS = 100_000
N_STOCKS = 1
TILL_MATURITY = 3.0
N_DAYS = 365 * TILL_MATURITY
RF_RATE = 0.03
VOL = 0.2

In [4]:
N_DAYS

1095.0

In [56]:
np.random.seed(RANDOM_SEED)

start = dt.datetime(2019, 1, 1)
end = start + dt.timedelta(days=N_DAYS)

random_returns = np.concatenate(
    [
        np.expand_dims(np.array([1.0]), 1),
        np.exp(
            (RF_RATE - VOL**2 / 2) / 252
            + VOL / np.sqrt(252) * np.random.randn((end - start).days, N_STOCKS)
        ),
    ],
    axis=0,
)
data = pd.DataFrame(
    random_returns,
    columns=["price"],
)
initial = data.copy()
data["price"] = data["price"].cumprod()
data["index"] = pd.date_range(start=start, end=end, freq="1D")
data.set_index("index", inplace=True)

underlyings = Underlyings(
    tickers=[Ticker(f"Stock {i + 1}", f"{i + 1}") for i in range(N_STOCKS)],
    start=start,
    end=end,
    data=data,
    dividends=np.array([0.0] * N_STOCKS),
)
underlyings.data

,price
index,
2019-01-01,1.000000
2019-01-02,1.006017
2019-01-03,0.997457
2019-01-04,1.000548
2019-01-05,0.979375
...,...
2021-12-27,0.979848
2021-12-28,0.990444
2021-12-29,0.984784


In [110]:
np.random.seed(RANDOM_SEED)
(VOL / np.sqrt(252) * np.random.randn(100_000, N_STOCKS)).std(ddof=0) * np.sqrt(252)

0.1994663037488388

In [100]:
np.random.seed(RANDOM_SEED)
r = np.random.randn((end - start).days, N_STOCKS)
r

array([[ 0.47298583],
       [-0.68142588],
       [ 0.2424395 ],
       ...,
       [-0.45801124],
       [ 1.28664739],
       [-1.98921027]])

In [104]:
r.mean()

-0.0048969702011397685

In [106]:
np.mean((r - r.mean()) ** 2) / d((end - start).days)

1.0514899320057765

In [99]:
d((end - start).days)

0.9997715069447598

In [72]:
np.log(random_returns[1:]).std() * np.sqrt(252)

0.20506093473488884

In [66]:
data / data.shift(1)

,price
index,
2019-01-01,NaN
2019-01-02,1.006017
2019-01-03,0.991491
2019-01-04,1.003099
2019-01-05,0.978840
...,...
2021-12-27,1.015071
2021-12-28,1.010814
2021-12-29,0.994286


In [78]:
np.log(data / data.shift(1)).std(ddof=0) * np.sqrt(252)

price    0.205061
dtype: float64

In [64]:
np.log(random_returns[1:]).std()

0.012917624688641965

In [58]:
np.sqrt(np.diag(underlyings.get_var_covar()))

array([0.20508802])

In [7]:
curve = ConstantRateCurve(rate=RF_RATE)

In [8]:
european_call = EuropeanCall(
    underlyings=underlyings,
    yield_curve=curve,
    strike_level=1.0,
    start_date=start,
    end_date=end,
)

In [9]:
pricer = GBMPricer(payoff_function=european_call.payoff, random_seed=RANDOM_SEED)

In [10]:
paths = pricer.get_paths(
    current_spot=[1.0],
    time_till_maturity=TILL_MATURITY,
    risk_free_rate_fn=curve.get_rate,
    dividends_fn=lambda t: 0.0,
    var_covar_fn=lambda t: np.array([VOL**2]),
    n_paths=N_PATHS,
)
paths.shape

(100000, 757, 1)

In [11]:
pricer.price(
    current_spot=[1.0],
    time_till_maturity=TILL_MATURITY,
    risk_free_rate_fn=curve.get_rate,
    dividends_fn=lambda t: 0.0,
    var_covar_fn=lambda t: np.array([VOL**2]),
    n_paths=N_PATHS,
)

0.17888212769640016

In [12]:
european_call.price()

[0.43096749]
[0.07571642]


array([0.18220811])

In [13]:
final_points = np.log(paths[:, -1, :].squeeze(1))
final_points.mean() / 3, final_points.std() / np.sqrt(3)

(0.010110012826833657, 0.19966178768086412)

In [14]:
underlyings.get_dividends()

array([0.])

In [15]:
np.maximum(paths[:, -1] - 1.0, 0).mean()

0.19572822393987263

In [16]:
curve.get_rate(3.0)

0.03